In [ ]:
from flask import Flask, render_template, request, redirect
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import tensorflow 
from tensorflow import keras
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [ ]:
model = keras.models.load_model('Spam Detector.h5')

In [ ]:
ps=PorterStemmer()
voc_size = 5000
nltk.download('stopwords')
sentence_length = 25
def prepro(message):
    review0 = re.sub('[^a-zA-Z]', ' ', message)
    review1 = review0.lower() 
    review2 = review1.split()
    review3 = [ps.stem(word) for word in review2 if not word in stopwords.words('english')]
    review4 = ' '.join(review3)           
    onehot= one_hot(review4, voc_size)
    onehot=np.reshape(onehot,(1,len(onehot)))
    embedded_docs = pad_sequences(onehot, padding = 'pre', maxlen= sentence_length)
    return embedded_docs


app = Flask(__name__)
@app.route('/')
def home():
    return render_template('home.html')


@app.route('/predict', methods =['POST'])
def predict():
    if request.method == 'POST':
        message = request.form['message']
        me=prepro(message)
        result = model.predict(me).round().flatten()[0]
    
        if result == 1:
            prediction= 'Oopss!!! This message is a SPAM'
            return render_template('predict.html', pred= prediction)
        else:
            prediction = 'Yayy!!! This message is not a SPAM'
            return render_template('predict.html', pred= prediction)  
        

if __name__=='__main__':
    app.run()
